In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
torch.cuda.empty_cache()
model_name = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, 
    attn_implementation="eager",  
    device_map={"": "cuda"}
)
processor = AutoProcessor.from_pretrained(model_name,use_fast=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kmano\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

RuntimeError: Failed to import transformers.models.auto.processing_auto because of the following error (look up to see its traceback):
numpy.core.umath failed to import

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "image": ""#your path,
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "Consider yourself as a airforce pilot who is operating a drone at this moment, explain this event."},
        ],
    }
]

In [ ]:
messages

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print("".join(output_text))

In [ ]:
print("".join(output_text))

In [ ]:
torch.cuda.empty_cache()

In [6]:
!pip install numpy

                                              0.0/12.9 MB ? eta -:--:--
                                              0.1/12.9 MB 1.7 MB/s eta 0:00:08
                                              0.2/12.9 MB 1.8 MB/s eta 0:00:07
                                              0.3/12.9 MB 2.4 MB/s eta 0:00:06
     --                                       0.7/12.9 MB 3.8 MB/s eta 0:00:04
     ---                                      1.1/12.9 MB 5.1 MB/s eta 0:00:03
     -----                                    1.8/12.9 MB 6.7 MB/s eta 0:00:02
     -------                                  2.5/12.9 MB 8.0 MB/s eta 0:00:02
     ---------                                3.2/12.9 MB 8.9 MB/s eta 0:00:02
     -------------                            4.3/12.9 MB 10.6 MB/s eta 0:00:01
     -----------------                        5.6/12.9 MB 12.2 MB/s eta 0:00:01
     -------------------                      6.5/12.9 MB 12.9 MB/s eta 0:00:01
     ---------------------                    7.1/12.9 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.73 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.2.4 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.2.4 which is incompatible.
streamlit 1.39.0 requires pillow<11,>=7.1.0, but you have pillow 11.0.0 which is incompatible.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.4 which is incompatible.

[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
